In [ ]:
%env CUDA_PATH=/usr/local/cuda

In [ ]:
import os
import numpy as np
import opendatasets as od
import matplotlib.pyplot as plt
from glob import glob
import pandas as pd
import json
import pickle

from ml_genn import InputLayer, Layer, SequentialNetwork, Network, Population, Connection
from ml_genn.callbacks import Checkpoint, SpikeRecorder, VarRecorder, Callback, OptimiserParamSchedule
from ml_genn.compilers import EventPropCompiler, InferenceCompiler
from ml_genn.connectivity import Dense
from ml_genn.initializers import Normal
from ml_genn.neurons import LeakyIntegrate, LeakyIntegrateFire, SpikeInput, LeakyIntegrateFireInput
from ml_genn.optimisers import Adam
from ml_genn.serialisers import Numpy
from ml_genn.synapses import Exponential
from time import perf_counter

from ml_genn.utils.data import (calc_latest_spike_time, linear_latency_encode_data)
from ml_genn.compilers.event_prop_compiler import default_params

from ml_genn.utils.data import (calc_latest_spike_time, calc_max_spikes,
                                preprocess_tonic_spikes)

In [ ]:
sample_id = 0

In [ ]:
# Kaggle dataset directory
dataset = 'https://www.kaggle.com/datasets/thomasshoesmith/spiking-google-speech-commands/data'

# Using opendatasets to download SGSC dataset
od.download(dataset)

x_test = np.load("spiking-google-speech-commands/testing_x_spikes.npy", allow_pickle=True)
y_test = np.load("spiking-google-speech-commands/testing_y_spikes.npy", allow_pickle=True)


with open("SC_params.json", "r") as f: 
    params = json.load(f)

schedule_epoch_total = 0

# READ TO BE IMPLEMENTED 
# Preprocess
x_test_spikes = []
for i in range(len(x_test)):
    events = x_test[i]
    x_test_spikes.append(preprocess_tonic_spikes(events, 
                                                  x_test[0].dtype.names,
                                                  (80, 1, 1),
                                                  time_scale = 1))

# Determine max spikes and latest spike time
max_spikes = calc_max_spikes(x_test_spikes)
latest_spike_time = calc_latest_spike_time(x_test_spikes)
print(f"Max spikes {max_spikes}, latest spike time {latest_spike_time}")

# Get number of input and output neurons from dataset 
# and round up outputs to power-of-two
num_input = 80 #int(np.prod(x.sensor_size))
num_output = 35 #len(x.classes)

os.chdir("output")

In [ ]:
dir = "~/PhD/Intel-Neuromorphic-Research-Project/output/full_100_epoch/"

output = pd.read_csv(os.path.expanduser(dir + "train_output.csv"))

with open(os.path.expanduser(dir + "params.json"), "r") as f:
        params = json.load(f)

# change dir for readout files
# lazy fix until a solution can be implemented with ml_genn to support output file directory change
try:
    os.makedirs(params.get("output_dir") + params.get("sweeping_suffix"))
except:
    pass

os.chdir(params.get("output_dir") + params.get("sweeping_suffix"))



serialiser = Numpy(os.path.expanduser(dir + "/SC"))
network = Network(default_params)
    
with network:
    # Populations
    input = Population(SpikeInput(max_spikes = params["BATCH_SIZE"] * max_spikes),
                       num_input,
                       record_spikes=True)
    
    hidden = Population(LeakyIntegrateFire(v_thresh=1.0, 
                                    tau_mem=20.0),
                params.get("NUM_HIDDEN"), 
                record_spikes=True)
    
    output = Population(LeakyIntegrate(tau_mem=20.0, 
                                readout="avg_var_exp_weight"),
                params.get("NUM_OUTPUT"), 
                record_spikes=True)

    Connection(input, hidden, Dense(Normal(mean = params.get("input_hidden_w_mean"), 
                                            sd = params.get("input_hidden_w_sd"))),
                Exponential(2.0))
    
    if params.get("recurrent"):
        Connection(hidden, hidden, Dense(Normal(mean = params.get("hidden_hidden_w_mean"), 
                                                sd = params.get("hidden_hidden_w_sd"))),
                Exponential(2.0))
    
    Connection(hidden, output, Dense(Normal(mean = params.get("hidden_output_w_mean"),
                                sd = params.get("hidden_output_w_sd"))),
                Exponential(2.0))

network.load((params.get("NUM_EPOCH") - 1,), serialiser)

compiler = InferenceCompiler(evaluate_timesteps = params.get("NUM_FRAMES") * params.get("INPUT_FRAME_TIMESTEP"),
                            reset_in_syn_between_batches=True,
                            #quantise_num_weight_bits=8,
                            #quantise_weight_percentile=99,
                            batch_size = params.get("BATCH_SIZE"))
compiled_net = compiler.compile(network)

with compiled_net:

    callbacks = [SpikeRecorder(input, 
                               key = "input_spikes",
                               example_filter = sample_id),
                 SpikeRecorder(hidden,
                               key = "hidden_spikes",
                               example_filter = sample_id),
                 VarRecorder(output, 
                             var = "v",
                             key = "output_voltages",
                             example_filter = sample_id)]

    metrics, cb_data = compiled_net.evaluate({input: x_test_spikes},
                                             {output: y_test},
                                             callbacks = callbacks)
    
    #compiled_net.save(("quant8",), serialiser)

    print(f"Accuracy = {100 * metrics[output].result}%")

In [ ]:
# Input spike activity
plt.title('Input Spikes (with raw input for comparison) from lava')

plt.scatter(cb_data["input_spikes"][0][0], cb_data["input_spikes"][1][0],
            s = 1)

# no clue why I need to rotate and flip...could be investigated? 

plt.ylim(0, 80)
plt.xlim(0, 2000)

plt.show()

In [ ]:
plt.scatter(cb_data["hidden_spikes"][0][0], cb_data["hidden_spikes"][1][0],
            s = 0.5)
plt.ylabel("neurons")
plt.xlabel("timesteps")
plt.title("Hidden layer activity")
plt.ylim(0, 512)
plt.xlim(0, 2000)
plt.show()

In [ ]:
plt.plot(cb_data["output_voltages"][0])
plt.ylabel("voltage (v)")
plt.xlabel("timesteps")
plt.title("output voltage activity")
plt.xlim(0, 2000)
plt.show()